Estimate a generalized additive regression model (GAM) to explain N400 amplitude on the given datasets.

In [1]:
import io
from itertools import product
import logging
L = logging.getLogger(__name__)
from pathlib import Path
import pickle
import yaml

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as st
import torch
from tqdm.auto import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("..")
from berp.datasets import BerpDataset, NestedBerpDataset, get_metadata
from berp.datasets.eeg import load_eeg_dataset
from berp.viz import make_word_onset_epochs

In [4]:
workflow = "heilbron2022"
lm = "EleutherAI/gpt-neo-2.7B/n10000"

subjects = list(range(1, 20))
runs = list(range(1, 20))
stories = ["old-man-and-the-sea"]

# target_sensors = ["B19", "B20", "B22"]
target_sensors = ['B22', 'D19', 'A19', 'C22', 'C10', 'C32', 'A7', 'B4']

normalize_X_ts = False
normalize_X_variable = False
normalize_Y = False

epoch_tmin = -12/128
epoch_tmax = 96/128

baseline_tmin = -12/128
baseline_tmax = 0.

temporal_rois = [("baseline", baseline_tmin, baseline_tmax),
                 ("n400", 0.3, 0.5)]

out_path = "n400_gam_data.csv"

# # Only run quantitative tests on items we have a priori interest in. We want to avoid
# # fishing for p-values here, and also degrading any positive results due to multiple
# # comparisons.
# # Semantics: (feature, troi, sensor) where we expect amplitude/latency-to-peak differences
# a_priori_interest = [
#     ("var_word_surprisal", "n125", "D19"),
#     ("var_word_surprisal", "n125", "C10"),
#     ("var_word_surprisal", "n400", "A19"),
#     ("var_word_surprisal", "early", "C10"),
# ]

In [5]:
dataset_dir = f"../workflow/{workflow}/data/dataset/{lm}"
stimulus_dir = f"../workflow/{workflow}/data/stimulus/{lm}"

## Prepare dataset

In [6]:
# Load datasets.
ds = {}
stimulus_paths = {f"{story}/run{run}": Path(stimulus_dir) / f"{story}/run{run}.pkl"
                  for story in stories for run in runs}
combs = list(product(stories, subjects, runs))
ds_paths = []
for story, subject, run in tqdm(combs):
    ds_path = Path(dataset_dir) / story / f"sub{subject}" / f"run{run}.pkl"
    if not ds_path.exists():
        print(f"Could not find dataset {ds_path}")
        continue
    ds_paths.append(ds_path)
    
nested_ds = load_eeg_dataset(
    ds_paths,
    subset_sensors=target_sensors,
    stimulus_paths=stimulus_paths,
    normalize_X_ts=normalize_X_ts, 
    normalize_X_variable=normalize_X_variable, 
    normalize_Y=normalize_Y)

assert len(combs) == len(nested_ds.datasets)
ds = dict(zip(combs, nested_ds.datasets))

  0%|          | 0/361 [00:00<?, ?it/s]

## Epoch

In [7]:
df = pd.concat([make_word_onset_epochs(
        ds_i,
        tmin=epoch_tmin, tmax=epoch_tmax,
        baseline=False,
    ) for ds_i in tqdm(ds.values())],
    names=["story", "subject", "run"],
    keys=ds.keys())
df.index = df.index.rename("word_idx", level="epoch")
df = df.drop(columns=["word_idx"])
df["sensor_name"] = df.index.get_level_values("sensor_idx").map(dict(enumerate(target_sensors)))
df

  0%|          | 0/361 [00:00<?, ?it/s]

value  \
story               subject run word_idx sample sensor_idx             
old-man-and-the-sea 1       1   0        0      0           0.031092   
                                                1           0.056925   
                                                2           0.044012   
                                                3           0.032322   
                                                4          -0.000803   
...                                                              ...   
                    19      19  588      107    3           0.026362   
                                                4           0.039911   
                                                5           0.054915   
                                                6          -0.127584   
                                                7          -0.161931   

                                                            epoch_time  \
story               subject run word_idx sample sensor_idx               
old-man-and-the-sea 1       1   0        0      0            -0.093750   
                                                1            -0.093750   
                                                2            -0.093750   
                                                3            -0.093750   
                                                4            -0.093750   
...                                                                ...   
                    19      19  588      107    3             0.742188   
                                                4             0.742188   
                                                5             0.742188   
                                                6             0.742188   
                                                7             0.742188   

                                                           sensor_name  
story               subject run word_idx sample sensor_idx              
old-man-and-the-sea 1       1   0        0      0                  B22  
                                                1                  D19  
                                                2                  A19  
                                                3                  C22  
                                                4                  C10  
...                                                                ...  
                    19      19  588      107    3                  C22  
                                                4                  C10  
                                                5                  C32  
                                                6                   A7  
                                                7                   B4  

[176439168 rows x 3 columns]

### Prepare design matrix

In [7]:
metadata_df = pd.concat([get_metadata(ds_i) for ds_i in ds.values()],
                        names=["story", "subject", "run"],
                        keys=ds.keys())
metadata_df

word_length  word_duration  \
story               subject run word_idx                               
old-man-and-the-sea 1       1   0                   3       0.156250   
                                1                   2       0.078125   
                                2                   3       0.289062   
                                3                   3       0.382812   
                                4                   2       0.210938   
...                                               ...            ...   
                    19      19  584                 4       0.250000   
                                585                 2       0.156250   
                                586                 2       0.109375   
                                587                 2       0.117188   
                                588                 7       0.632812   

                                          word_surprisal  word_frequency  \
story               subject run word_idx                                   
old-man-and-the-sea 1       1   0              -2.381337        7.429644   
                                1              -3.896170        9.030593   
                                2              -3.420624       10.644726   
                                3              -0.872183        9.044897   
                                4              -1.986570        8.776630   
...                                                  ...             ...   
                    19      19  584            -4.542065       12.977191   
                                585            -1.088804        8.243340   
                                586            -0.945295        5.689068   
                                587            -2.847154        6.640238   
                                588            -3.234899       18.427759   

                                          word_prior_entropy     word  
story               subject run word_idx                               
old-man-and-the-sea 1       1   0                   5.486012      wʌz  
                                1                   5.764207       ʌn  
                                2                   6.296887     oʊld  
                                3                   4.189036      mæn  
                                4                   3.307850       hu  
...                                                      ...      ...  
                    19      19  584                 4.395442     lʊkt  
                                585                 2.250363       æt  
                                586                 1.269512       ɪt  
                                587                 2.992393       ɪn  
                                588                 3.286217  dɪsgʌst  

[204212 rows x 6 columns]

## Aggregate

In [9]:
# Identifies a single epoch
id_key = ["story", "subject", "run", "word_idx", "sensor_name"]

In [10]:
roi_df = {}

for label, tstart, tend in temporal_rois:
    sub_comb_df = df[df.epoch_time.between(tstart, tend)]
    roi_df[label] = sub_comb_df.groupby(id_key).value.mean()
    
roi_df = pd.concat(roi_df, names=["toi"])
roi_df

toi       story                subject  run  word_idx  sensor_name
baseline  old-man-and-the-sea  1        1    0         A19            0.041212
                                                       A7             0.107339
                                                       B22            0.026291
                                                       B4             0.105514
                                                       C10            0.010029
                                                                        ...   
n400      old-man-and-the-sea  19       19   588       B4            -0.179150
                                                       C10           -0.026348
                                                       C22           -0.246394
                                                       C32           -0.053439
                                                       D19           -0.223328
Name: value, Length: 3267392, dtype: float64

In [11]:
agg_df = roi_df.unstack("toi")
agg_df

toi                                                   baseline      n400
story               subject run word_idx sensor_name                    
old-man-and-the-sea 1       1   0        A19          0.041212 -0.002146
                                         A7           0.107339  0.094173
                                         B22          0.026291  0.006058
                                         B4           0.105514  0.067242
                                         C10          0.010029  0.034790
...                                                        ...       ...
                    19      19  588      B4           0.202290 -0.179150
                                         C10          0.026011 -0.026348
                                         C22          0.012474 -0.246394
                                         C32          0.021017 -0.053439
                                         D19          0.107252 -0.223328

[1633696 rows x 2 columns]

In [12]:
full_df = pd.merge(agg_df, metadata_df, left_index=True, right_index=True)
full_df

baseline      n400  \
story               subject run word_idx sensor_name                       
old-man-and-the-sea 1       1   0        A19          0.041212 -0.002146   
                                         A7           0.107339  0.094173   
                                         B22          0.026291  0.006058   
                                         B4           0.105514  0.067242   
                                         C10          0.010029  0.034790   
...                                                        ...       ...   
                    19      19  588      B4           0.202290 -0.179150   
                                         C10          0.026011 -0.026348   
                                         C22          0.012474 -0.246394   
                                         C32          0.021017 -0.053439   
                                         D19          0.107252 -0.223328   

                                                      word_length  \
story               subject run word_idx sensor_name                
old-man-and-the-sea 1       1   0        A19                    3   
                                         A7                     3   
                                         B22                    3   
                                         B4                     3   
                                         C10                    3   
...                                                           ...   
                    19      19  588      B4                     7   
                                         C10                    7   
                                         C22                    7   
                                         C32                    7   
                                         D19                    7   

                                                      word_duration  \
story               subject run word_idx sensor_name                  
old-man-and-the-sea 1       1   0        A19               0.156250   
                                         A7                0.156250   
                                         B22               0.156250   
                                         B4                0.156250   
                                         C10               0.156250   
...                                                             ...   
                    19      19  588      B4                0.632812   
                                         C10               0.632812   
                                         C22               0.632812   
                                         C32               0.632812   
                                         D19               0.632812   

                                                      word_surprisal  \
story               subject run word_idx sensor_name                   
old-man-and-the-sea 1       1   0        A19               -2.381337   
                                         A7                -2.381337   
                                         B22               -2.381337   
                                         B4                -2.381337   
                                         C10               -2.381337   
...                                                              ...   
                    19      19  588      B4                -3.234899   
                                         C10               -3.234899   
                                         C22               -3.234899   
                                         C32               -3.234899   
                                         D19               -3.234899   

                                                      word_frequency  \
story               subject run word_idx sensor_name                   
old-man-and-the-sea 1       1   0        A19                7.429644   
                                         A7                 7.429644   
                  

In [13]:
# DEV for now export to do it in R vanilla :/
full_df.to_csv(out_path)